# **Preparing Data**

In [ ]:
import pandas as pd

#Load Dataset
dataset=pd.read_csv("C:/Users/Wintest/Desktop/FakenewsNet/Complete.csv")

dataset = dataset.astype(dtype={'title': 'string',
                                'author_name': 'string',
                                'content': 'string','label': 'string'})


In [ ]:
dataset.label.value_counts()

In [ ]:
duplicateRowsDF = dataset[dataset.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

# **Semantic Analysis**

## **Quantity Analysis**

In [ ]:
#Calculate Number of Postive and Negative Words

PosWords = open('L:/Thesis/Dateset/PostiveWord.txt' , 'r')
NegWords = open('L:/Thesis/Dateset/NegativeWords.txt' , 'r')

PosWordSet = list (PosWords)
NegWordSet = list (NegWords)

countOfPosWord = list()
countOfNegWord = list()


for row in range(0,len(dataset)):
  NumOfPosWord=0
  word = dataset['content'][row].split()
  for i in range(0,len(PosWordSet)):
    PosWordSet[i] = PosWordSet[i].replace('\n','')
    for j in range(0,len(word)):
      if word[j] == PosWordSet[i]:
        NumOfPosWord +=1
  countOfPosWord.insert(row,NumOfPosWord)
  NumOfNegWord=0
  word = dataset['content'][row].split()
  for i in range(0,len(NegWordSet)):
    NegWordSet[i] = NegWordSet[i].replace('\n','')
    for j in range(0,len(word)):
      if word[j] == NegWordSet[i]:
        NumOfNegWord +=1
  countOfNegWord.insert(row,NumOfNegWord)

dataset['countOfPosWord'] = countOfPosWord
dataset['countOfNegWord'] = countOfNegWord

In [ ]:
!pip install textstat

In [ ]:
NumOfChar = list()
NumOfWord = list()
NumOfSen = list()

In [ ]:
import textstat
#Calculate number of characters

for row in range(0,len(dataset)):
  NumOfChar.insert(row,textstat.char_count(dataset['content'][row]))

#Calculate number of words
for row in range(0,len(dataset)):
  NumOfWord.insert(row,textstat.lexicon_count(dataset['content'][row]))

#Calculate number of sentences

for row in range(0,len(dataset)):
  NumOfSen.insert(row,textstat.sentence_count(dataset['content'][row]))

dataset['NumberOfChar']=NumOfChar
dataset['NumberOfWords']=NumOfWord
dataset['NumberOfSentences']=NumOfSen

AvgCharPerWord=list()
AvgWordPerSent=list()

for row in range(0,len(dataset)):
  AvgCharPerWord.insert(row,textstat.avg_character_per_word(dataset['content'][row]))

for i in range(0,len(NumOfChar)):
    AvgWordPerSent.insert(i,(NumOfWord[i] / NumOfSen[i]))


dataset['AvgCharPerWord']=AvgCharPerWord
dataset['AvgWordPerSent']=AvgWordPerSent

NumberOfUpCase=list()
for row in range(0,len(dataset)):
  uppercase_count=0  
  for character in dataset['content'][row]:
      if character.isupper():
          uppercase_count += 1
  NumberOfUpCase.insert(row,uppercase_count)
print(row)
dataset['NumberOfUpCase']=NumberOfUpCase

#Calculate Number of punctuations
import string
puncsigns=string.punctuation
NumOfPunc=list()
for row in range(0,len(dataset)):
  Punc=0
  for char in dataset['content'][row]:
    if char in puncsigns:
      Punc = Punc+1
  NumOfPunc.insert(row,Punc)
dataset['NumberOfPunctuatuion']=NumOfPunc



In [ ]:
dataset

In [ ]:
from google.colab import files

dataset.to_csv('Export_dataset.csv')


## **Topic Modelling**

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Thesis Datasets/Export_dataset.csv")
dataset.set_index('id')

dataset.info()

In [ ]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
import random
from gensim import corpora
import pickle
import gensim

def topicModelling(content):
  text_data = []
  while(True):
      tokens = prepare_text_for_lda(content)
      if random.random() > .99:
          #print(tokens)
          text_data.append(tokens)
          break;
  dictionary = corpora.Dictionary(text_data)
  corpus = [dictionary.doc2bow(text) for text in text_data]
  pickle.dump(corpus, open('corpus.pkl', 'wb'))
  dictionary.save('c:/users/wintest/desktop/dictionary.gensim')
  NUM_TOPICS = 10
  if len(corpus[0]) == 0:
    return ('none')
  else:
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    my_dict = {'Topic_' + str(i): [token for token, score in ldamodel.show_topic(i, topn=10)] for i in range(0, ldamodel.num_topics)}
    print(my_dict['Topic_9'])
  
    return(my_dict['Topic_9'])

In [ ]:
contentTopicModelling = list()

for row in range(0,len(dataset)):
  print(row)

  topicmodelling = topicModelling(dataset['content'][row])

  if topicmodelling  == 'none':
    contentTopicModelling.insert(row,'Null')
    #dataset = dataset.drop(dataset.index[row])
    print('Deleted empty row')
    
  else:
    contentTopicModelling.insert(row,topicmodelling)

dataset['ContentTM'] = contentTopicModelling

In [ ]:

dataset.to_csv('Export_dataset2.csv')

## **Semantic Analysis**

In [ ]:
import pandas as pd
dataset = pd.read_csv("D:/Rezaei/Export_dataset2.csv")
dataset.set_index('id')
dataset = dataset.astype(dtype={'title': 'string','url': 'string',
                                'date_published': 'string','author_name': 'string',
                                'content': 'string','label': 'string','ContentTM':'string'})

dataset.info()

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

global documents_df
#tfidfvectoriser=int()
#vocab_size=int()

In [ ]:
def preprocssing(documents_df):
  stop_words_l=stopwords.words('english')
  documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

In [ ]:
def TfidfSim(documents_df):
    global tfidfvectoriser
    tfidfvectoriser=TfidfVectorizer(max_features=64)
    tfidfvectoriser.fit(documents_df.documents_cleaned)
    global tfidf_vectors
    tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)
    tfidf_vectors=tfidf_vectors.toarray()
    pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
    return(pairwise_similarities[0][1])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import gensim
from gensim import models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def word2vecSim(documents_df):
  global tokenizer
  tokenizer=Tokenizer()
  tokenizer.fit_on_texts(documents_df.documents_cleaned)
  tokenized_documents=tokenizer.texts_to_sequences(documents_df.documents_cleaned)
  global tokenized_paded_documents
  tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=64,padding='post')
  global vocab_size
  vocab_size=len(tokenizer.word_index)+1
  # loading pre-trained embeddings, each word is represented as a 300 dimensional vector

  

In [ ]:
def gloveSim(documents_df):
  # reading Glove word embeddings into a dictionary with "word" as key and values as word vectors

  embeddings_index = dict()

  with open('G:/datasets/glove.6B/glove.6B.50d.txt', encoding='utf-8') as file:
      for line in file:
          values = line.split()
          word = values[0]
          coefs = np.asarray(values[1:], dtype='float64')
          embeddings_index[word] = coefs
  # creating embedding matrix, every row is a vector representation from the vocabulary indexed by the tokenizer index. 

  embedding_matrix=np.zeros((vocab_size,50))

  for word,i in tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector
  # tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents

  document_embeddings=np.zeros((len(tokenized_paded_documents),50))
  words=tfidfvectoriser.get_feature_names()

  for i in range(documents_df.shape[0]):
      for j in range(len(words)):
          document_embeddings[i]+=embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
  document_embeddings=document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
  pairwise_similarities=cosine_similarity(document_embeddings)
  return(pairwise_similarities[0][1])


In [ ]:
!pip install sentence_transformers

In [ ]:

from sentence_transformers import SentenceTransformer
def bertSim(documents_df):
  sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  document_embeddings = sbert_model.encode(documents_df['documents_cleaned'])
  pairwise_similarities=cosine_similarity(document_embeddings)
  return(pairwise_similarities[0][1])


In [ ]:
len(dataset)

### **Main**

In [ ]:
#data=pd.read_csv('/content/drive/MyDrive/Thesis Datasets/T1.csv')
similarity=list()
for row in range(0,len(dataset)):
  try:
    documents = [dataset['title'][row],dataset['content'][row],]
    pd.set_option('display.max_colwidth', 0)
    pd.set_option('display.max_columns', 0)
    documents_df=pd.DataFrame(documents,columns=['documents'])
    preprocssing(documents_df)
    tfidfsim=TfidfSim(documents_df)
    word2vecsim=word2vecSim(documents_df)
    glovesim=gloveSim(documents_df)
    bertsim=bertSim(documents_df)
    avgSimilarity=(tfidfsim+glovesim+bertsim)/3
    print(avgSimilarity,"-->",row)
    similarity.insert(row,avgSimilarity)
  except ValueError:
    similarity.insert(row,"ValueError")
dataset['Orgtitle_contentSim']=similarity

In [ ]:
dataset.to_csv('Export_dataset3.csv')

In [ ]:
similarityTM=list()
for row in range(0,len(dataset)):
  try:
    documents = [dataset['title'][row],dataset['ContentTM'][row],]
    pd.set_option('display.max_colwidth', 0)
    pd.set_option('display.max_columns', 0)
    documents_df=pd.DataFrame(documents,columns=['documents'])
    preprocssing(documents_df)
    tfidfsim=TfidfSim(documents_df)
    word2vecsim=word2vecSim(documents_df)
    glovesim=gloveSim(documents_df)
    bertsim=bertSim(documents_df)
    avgSimilarity=(tfidfsim+glovesim+bertsim)/3
    print(avgSimilarity,"-->",row)
    similarityTM.insert(row,avgSimilarity)
  except ValueError:
    similarityTM.insert(row,"ValueError")
dataset['TopicModellingSim'] = similarityTM

In [ ]:
dataset.to_csv('Export_dataset10.csv')

# **Sentiment Analysis**

In [ ]:

!pip install afinn

In [ ]:
from afinn import Afinn
afinn = Afinn(language='en')

In [ ]:
dataset['afinn_score'] = dataset['content'].apply(afinn.score)

In [ ]:
dataset

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
dataset

In [ ]:
def vaderize(df, textfield):
    '''Compute the Vader polarity scores for a textfield.
    Returns scores and original dataframe.'''

    analyzer = SentimentIntensityAnalyzer()

    print('Estimating polarity scores for %d cases.' % len(df))
    sentiment = df[textfield].apply(analyzer.polarity_scores)

    # convert to dataframe
    sdf = pd.DataFrame(sentiment.tolist()).add_prefix('vader_score_')

    # merge dataframes
    df_combined = pd.concat([df, sdf], axis=1)
    del df_combined['vader_score_neg']
    del df_combined['vader_score_neu']
    del df_combined['vader_score_pos']

    return df_combined

In [ ]:
dataset=vaderize(dataset,'content')

In [ ]:
dataset

In [ ]:


dataset.to_csv('Export_dataset5.csv')

# **Readability**

In [ ]:
import textstat

#Flesch Reading Ease Index
FREI=list()

#Flesch-Kionciad Grade Level
FKGL=list()

#Automated Readability Index
ARI=list()

#Gunning Fox Index
GFI=list()

#Coleman-Liau Index
CLI=list()

for row in range(0,len(dataset)):
  FREI.insert(row,(textstat.flesch_reading_ease(dataset['content'][row])))
  FKGL.insert(row,(textstat.flesch_kincaid_grade(dataset['content'][row])))
  ARI.insert(row,(textstat.automated_readability_index(dataset['content'][row])))
  GFI.insert(row,(textstat.gunning_fog(dataset['content'][row])))
  CLI.insert(row,(textstat.coleman_liau_index(dataset['content'][row])))

dataset['FREI'] = FREI
dataset['FKGL'] = FKGL
dataset['ARI'] = ARI
dataset['GFI'] = GFI
dataset['CLI'] = CLI



In [ ]:


dataset.to_csv('Export_dataset6.csv')